In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
pool_info = {
    "../../datasets/sol_pengu_pool/sol_pengu_pool_swaps.csv": {
        "pool_address": "FAqh648xeeaTqL7du49sztp9nfj5PjRQrfvaMccyd9cz",
        "fee_tier": 0.003
    },
    "../../datasets/msol_mnde_pool/msol_mnde_pool_swaps.csv": {
        "pool_address": "BVXNG6BrL2Tn3NmppnMeXHjBHTaQSnSnLE99JKwZSWPg",
        "fee_tier": 0.01
    },
    "../../datasets/sol_hnt_pool/sol_hnt_pool_swaps.csv": {
        "pool_address": "CSP4RmB6kBHkKGkyTnzt9zYYXDA8SbZ5Do5WfZcjqjE4",
        "fee_tier": 0.0005
    },
    "../../datasets/sol_usdc_pool/sol_usdc_pool_swaps.csv": {
        "pool_address": "Czfq3xZZDmsdGdUyrNLtRhGc47cXcZtLG4crryfu44zE",
        "fee_tier": 0.0004
    },
    "../../datasets/pyusd_usdc_pool/pyusd_usdc_pool_swaps.csv": {
        "pool_address": "9tXiuRRw7kbejLhZXtxDxYs2REe43uH2e7k1kocgdM9B",
        "fee_tier": 0.0001
    },
    "../../datasets/drift_jitosol_pool/drift_jitosol_pool_swaps.csv": {
        "pool_address": "7u3wk63dbFfN6WUdxpJ6SDNMwDixK1ti2J3Q21ws5Vxs",
        "fee_tier": 0.0016
    },
    "../../datasets/fart_usdc_pool/fart_usdc_with_price_final.csv": {
        "pool_address": "J5jzvT22u1Mt6de4gkBhEsTSTjBfYS7A6aF5jzu9ihkC",
        "fee_tier": 0.0005
    },
    "../../datasets/sol_weth_pool/sol_weth_pool_swaps.csv": {
        "pool_address": "HktfL7iwGKT5QHjywQkcDnZXScoh811k7akrMZJkCcEF",
        "fee_tier": 0.0005
    },

}

In [3]:
def add_pre_aggregation_features(df: pd.DataFrame, pool_address: str, fee_tier: float):
    df['pool_address'] = pool_address
    df['fee_tier'] = fee_tier
    df['token_amount_a_ui'] = df['token_amount_a'] / 10 ** df['decimals_a']
    df['token_amount_b_ui'] = df['token_amount_b'] / 10 ** df['decimals_b']
    df['volume_usd'] = df['token_amount_a_ui'] * df['token_price_a']
    df['fee_usd'] = df['volume_usd'] * df['fee_tier']
    df['lp_fee_usd'] = df['fee_usd'] * 0.87
    df['date'] = pd.to_datetime(df['block_time'], unit='s')
    df['price_ratio'] = df['token_price_a'] / df['token_price_b']
    df['tvl_usd'] = (df['post_balance_a'] * df['token_price_a'] + df['post_balance_b'] * df['token_price_b'])
    df['tvl_utilization'] = df['volume_usd'] / df['tvl_usd']
    df['balance_ratio'] = (df['post_balance_a'] * df['token_price_a']) / (df['post_balance_b'] * df['token_price_b'])
    df['balance_imbalance'] = abs(df['balance_ratio'] - 1.0)

    # df = df.sort_values(by='block_time', ascending=True)
    # df = df.drop(columns=['slot', 'block_time', 'tx_signature'])
    return df

def aggregate_date(df: pd.DataFrame):
    df = df.resample('1min', on='date').agg(
        pool_address=('pool_address', 'first'),
        fee_tier=('fee_tier', 'first'),
        token_mint_a=('token_mint_a', 'first'),
        token_mint_b=('token_mint_b', 'first'),
        token_vault_a=('token_vault_a', 'first'),
        token_vault_b=('token_vault_b', 'first'),

        num_swaps=('num_swaps', 'sum'),
        volume_usd=('volume_usd', 'sum'),
        fee_usd=('fee_usd', 'sum'),
        lp_fee_usd=('lp_fee_usd', 'sum'),

        token_amount_a=('token_amount_a_ui', 'sum'),
        token_amount_b=('token_amount_b_ui', 'sum'),
        pre_balance_a=('pre_balance_a', 'first'),
        pre_balance_b=('pre_balance_b', 'first'),
        post_balance_a=('post_balance_a', 'last'),
        post_balance_b=('post_balance_b', 'last'),

        tvl_usd_start=('tvl_usd', 'first'),
        tvl_usd_end=('tvl_usd', 'last'),
        tvl_utilization_start=('tvl_utilization', 'first'),
        tvl_utilization_end=('tvl_utilization', 'last'),
        balance_ratio_start=('balance_ratio', 'first'),
        balance_ratio_end=('balance_ratio', 'last'),
        balance_imbalance_start=('balance_imbalance', 'first'),
        balance_imbalance_end=('balance_imbalance', 'last'),

        token_price_a_start=('token_price_a', 'first'),
        token_price_a_end=('token_price_a', 'last'),
        token_price_b_start=('token_price_b', 'first'),
        token_price_b_end=('token_price_b', 'last'),
        token_ema_a_start=('token_ema_a', 'first'),
        token_ema_a_end=('token_ema_a', 'last'),
        token_ema_b_start=('token_ema_b', 'first'),
        token_ema_b_end=('token_ema_b', 'last'),
        price_ratio_start=('price_ratio', 'first'),
        price_ratio_end=('price_ratio', 'last')
    )
    
    return df


def post_aggregation_features(df: pd.DataFrame):
    df['pct_change_token_a'] = (df['token_price_a_end'] - df['token_price_a_start']) / df['token_price_a_start']
    df['pct_change_token_b'] = (df['token_price_b_end'] - df['token_price_b_start']) / df['token_price_b_start']

    df['price_ema_deviation_a'] = (df['token_price_a_end'] - df['token_ema_a_end']) / df['token_ema_a_end']
    df['price_ema_deviation_b'] = (df['token_price_b_end'] - df['token_ema_b_end']) / df['token_ema_b_end']

    # window = 5
    # df['vol_token_a'] = df['pct_change_token_a'].rolling(window).std()
    # df['vol_token_b'] = df['pct_change_token_b'].rolling(window).std()
    # df['vol_ratio'] = df['vol_token_a'] / (df['vol_token_b'] + 1e-8)

    df['price_ratio_change'] = (df['price_ratio_end'] - df['price_ratio_start']) / df['price_ratio_start']

    df['utilization_a'] = abs(df['token_amount_a']) / df['pre_balance_a']
    df['utilization_b'] = abs(df['token_amount_b']) / df['pre_balance_b']
    df['tvl_change'] = (df['tvl_usd_end'] - df['tvl_usd_start']) / df['tvl_usd_start']

    df['volume_per_swap'] = df['volume_usd'] / df['num_swaps']

    V_hodl = df['pre_balance_a'] * df['token_price_a_end'] + df['pre_balance_b'] * df['token_price_b_end']
    V_lp = df['post_balance_a'] * df['token_price_a_end'] + df['post_balance_b'] * df['token_price_b_end']
    df['impermanent_loss'] = V_lp / V_hodl - 1

    df['avg_swap_size_a'] = df['token_amount_a'] / df['num_swaps']
    df['avg_swap_size_b'] = df['token_amount_b'] / df['num_swaps']

    df['fee_revenue_per_swap'] = df['lp_fee_usd'] / df['num_swaps']
    df['fee_efficiency'] = df['lp_fee_usd'] / (df['volume_usd'] + 1e-8)
    df['revenue_per_liquidity'] = df['lp_fee_usd'] / df['tvl_usd_start']

    return df

In [4]:
df_all = pd.DataFrame()
for pool, info in pool_info.items():
    df = pd.read_csv(pool)
    df = add_pre_aggregation_features(df, info['pool_address'], info['fee_tier'])
    # df = aggregate_date(df)
    # df = post_aggregation_features(df)
    df_all = pd.concat([df_all, df])

In [ ]:
df_all[df_all['']]

In [10]:
df_all.to_csv("pool_swaps.csv", index=False)

In [76]:
df_all.dtypes

pool_address                object
fee_tier                   float64
token_mint_a                object
token_mint_b                object
token_vault_a               object
token_vault_b               object
num_swaps                    int64
volume_usd                 float64
fee_usd                    float64
lp_fee_usd                 float64
token_amount_a             float64
token_amount_b             float64
pre_balance_a              float64
pre_balance_b              float64
post_balance_a             float64
post_balance_b             float64
tvl_usd_start              float64
tvl_usd_end                float64
tvl_utilization_start      float64
tvl_utilization_end        float64
balance_ratio_start        float64
balance_ratio_end          float64
balance_imbalance_start    float64
balance_imbalance_end      float64
token_price_a_start        float64
token_price_a_end          float64
token_price_b_start        float64
token_price_b_end          float64
token_ema_a_start   

In [67]:
df_all[df_all['num_swaps'] == 0].shape

(1408304, 52)

In [68]:
df_clean = df_all[df_all['num_swaps'] != 0]

In [69]:
df_clean.shape

(71022, 52)

In [71]:
df_clean.head(50)

,pool_address,fee_tier,token_mint_a,token_mint_b,token_vault_a,token_vault_b,num_swaps,volume_usd,fee_usd,lp_fee_usd,token_amount_a,token_amount_b,pre_balance_a,pre_balance_b,post_balance_a,post_balance_b,tvl_usd_start,tvl_usd_end,tvl_utilization_start,tvl_utilization_end,balance_ratio_start,balance_ratio_end,balance_imbalance_start,balance_imbalance_end,token_price_a_start,token_price_a_end,token_price_b_start,token_price_b_end,token_ema_a_start,token_ema_a_end,token_ema_b_start,token_ema_b_end,price_ratio_start,price_ratio_end,pct_change_token_a,pct_change_token_b,price_ema_deviation_a,price_ema_deviation_b,vol_token_a,vol_token_b,vol_ratio,price_ratio_change,utilization_a,utilization_b,tvl_change,volume_per_swap,impermanent_loss,avg_swap_size_a,avg_swap_size_b,fee_revenue_per_swap,fee_efficiency,revenue_per_liquidity
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-10-07 14:05:20,FAqh648xeeaTqL7du49sztp9nfj5PjRQrfvaMccyd9cz,0.003,So11111111111111111111111111111111111111112,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,J757hq9DXGPDYfCoeGpTcD9A71NFgNqBRMXHrdVGyRxK,SdFLxX6sWTkKWje3Xb4YNewbm5ieaj3tfEJYeLTyqyg,1,93.073522,0.279221,0.242922,0.410538,2921.291527,15908.116922,1.752807e+08,15907.706385,1.752836e+08,9.193660e+06,9.193660e+06,1.012366e-05,1.012366e-05,0.645485,0.645485,0.354515,0.354515,226.711212,226.711212,0.031875,0.031875,229.867963,229.867963,0.032516,0.032516,7112.459524,7112.459524,0.000000,0.000000,-0.013733,-0.019714,NaN,NaN,NaN,0.000000,2.580681e-05,1.666636e-05,0.000000e+00,93.073522,4.708472e-09,0.410538,2921.291527,0.242922,0.00261,2.642276e-08
2025-10-07 14:06:00,FAqh648xeeaTqL7du49sztp9nfj5PjRQrfvaMccyd9cz,0.003,So11111111111111111111111111111111111111112,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,J757hq9DXGPDYfCoeGpTcD9A71NFgNqBRMXHrdVGyRxK,SdFLxX6sWTkKWje3Xb4YNewbm5ieaj3tfEJYeLTyqyg,2,1466.499968,4.399500,3.827565,6.462820,45999.465508,15880.955983,1.751456e+08,15880.955983,1.751456e+08,9.190345e+06,9.190344e+06,8.885015e-05,7.071949e-05,0.644787,0.645027,0.355213,0.354973,226.913337,226.913337,0.031898,0.031898,229.828063,229.828063,0.032509,0.032509,7113.781433,7113.781433,0.000000,0.000000,-0.012682,-0.018790,NaN,NaN,NaN,0.000000,4.069541e-04,2.626356e-04,-5.641868e-08,733.249984,0.000000e+00,3.231410,22999.732754,1.913782,0.00261,4.164768e-07
2025-10-07 14:06:10,FAqh648xeeaTqL7du49sztp9nfj5PjRQrfvaMccyd9cz,0.003,So11111111111111111111111111111111111111112,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,J757hq9DXGPDYfCoeGpTcD9A71NFgNqBRMXHrdVGyRxK,SdFLxX6sWTkKWje3Xb4YNewbm5ieaj3tfEJYeLTyqyg,9,2741.573355,8.224720,7.155506,12.083554,86063.661048,15877.357413,1.751712e+08,15865.273859,1.752573e+08,9.187436e+06,9.182036e+06,3.212309e-05,5.138837e-05,0.644987,0.644640,0.355013,0.355360,226.903005,226.849500,0.031882,0.031856,229.826165,229.819213,0.032508,0.032507,7116.953763,7121.078376,-0.000236,-0.000815,-0.012922,-0.020022,NaN,NaN,NaN,0.000580,7.610557e-04,4.913117e-04,-5.877664e-04,304.619262,5.456378e-08,1.342617,9562.629005,0.795056,0.00261,7.788361e-07
2025-10-07 14:07:40,FAqh648xeeaTqL7du49sztp9nfj5PjRQrfvaMccyd9cz,0.003,So11111111111111111111111111111111111111112,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,J757hq9DXGPDYfCoeGpTcD9A71NFgNqBRMXHrdVGyRxK,SdFLxX6sWTkKWje3Xb4YNewbm5ieaj3tfEJYeLTyqyg,1,559.692930,1.679079,1.460799,2.467044,17580.545456,15865.266466,1.752572e+08,15883.359266,1.754293e+08,9.191960e+06,9.192147e+06,0.000000e+00,6.088816e-05,0.644940,0.644767,0.355060,0.355233,226.865129,226.867810,0.031857,0.031857,229.748253,229.747494,0.032493,0.032492,7121.448284,7121.349135,0.000012,0.000026,-0.012534,-0.019544,NaN,NaN,NaN,-0.000014,1.554997e-04,1.003128e-04,2.032259e-05,559.692930,1.043908e-03,2.467044,17580.545456,1.460799,0.00261,1.589213e-07
2025-10-07 14:07:50,FAqh648xeeaTqL7du49sztp9nfj5PjRQrfvaMccyd9cz,0.003,So11111111111111111111111111111111111111112,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,J757hq9DXGPDYfCoeGpTcD9A71NFgNqBRMXHrdVGyRxK,SdFLxX6sWTkKWje3Xb4YNew

In [73]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71022 entries, 2025-10-07 14:05:20 to 2025-10-13 03:24:10
Data columns (total 52 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   pool_address             71022 non-null  object 
 1   fee_tier                 71022 non-null  float64
 2   token_mint_a             71022 non-null  object 
 3   token_mint_b             71022 non-null  object 
 4   token_vault_a            71022 non-null  object 
 5   token_vault_b            71022 non-null  object 
 6   num_swaps                71022 non-null  int64  
 7   volume_usd               71022 non-null  float64
 8   fee_usd                  71022 non-null  float64
 9   lp_fee_usd               71022 non-null  float64
 10  token_amount_a           71022 non-null  float64
 11  token_amount_b           71022 non-null  float64
 12  pre_balance_a            71022 non-null  float64
 13  pre_balance_b            71022 non-null  